## Connect via Python and Create a new database named `mydb`

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine

POSTGRES_USERNAME = ''
POSTGRES_PASSWORD = ''
POSTGRES_ENDPOINT = ''
POSTGRES_DATABASE = ''

CONN = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ENDPOINT}:5432/{POSTGRES_DATABASE}"

engine = create_engine(CONN)

conn = engine.connect()

try: 
    conn.execution_options(isolation_level="AUTOCOMMIT").execute("CREATE DATABASE mydb")
except Exception as e:
    print(e)
finally:
    conn.close()

## Create the following table schema into the database

| field name | data type |
| ---------- | --------- |
| student_id | integer |
| name | varchar |
| age | integer |
| gender | varchar |
| subject | varchar |
| marks | integer |

In [ ]:
conn = engine.connect()

try: 
    conn.execute("CREATE TABLE IF NOT EXISTS students (student_id int, name varchar,\
    age int, gender varchar, subject varchar, marks int);")
except Exception as e: 
    print("Error: Issue creating table")
    print(e)
finally:
    conn.close()

## Insert the following two rows in the table

- First Row:  1, "Raj", 23, "Male", "Python", 85
- Second Row: 2, "Priya", 22, "Female", "Python", 86

In [ ]:
conn = engine.connect()

try: 
    conn.execute("INSERT INTO students (student_id, name, age, gender, subject, marks) \
                 VALUES (%s, %s, %s, %s, %s, %s)", \
                 (1, "Raj", 23, "Male", "Python", 85))
    conn.execute("INSERT INTO students (student_id, name, age, gender, subject, marks) \
                  VALUES (%s, %s, %s, %s, %s, %s)",
                  ( 2, "Priya", 22, "Female", "Python", 86))
except Exception as e: 
    print("Error: Inserting Rows")
    print (e)
finally:
    conn.close()

## Retrieve the results and print

### **Method 1: Using connection fetch method**

In [ ]:
conn = engine.connect()

try: 
    result = conn.execute("SELECT * FROM students;")
except Exception as e: 
    print("Error: select *")
    print(e)

row = result.fetchone()
while row:
    print(row)
    row = result.fetchone()

### **Method 2: Using Pandas**

In [ ]:
conn = engine.connect()
pd.read_sql("SELECT * FROM students", con=conn)

## Perform CRUD operations on a sample music data

In [ ]:
query = """
CREATE TABLE Music (
    Artist VARCHAR(20) NOT NULL, 
    SongTitle VARCHAR(30) NOT NULL,
    AlbumTitle VARCHAR(25),
    Year INT,
    Price FLOAT,
    Genre VARCHAR(10),
    CriticRating FLOAT,
    Tags TEXT,
    PRIMARY KEY(Artist, SongTitle)
);
"""

In [ ]:
conn = engine.connect()

try: 
    result = conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Year, Price, Genre, CriticRating, 
    Tags)
VALUES(
    'No One You Know', 'Call Me Today', 'Somewhat Famous',
    2015, 2.14, 'Country', 7.8,
    '{"Composers": ["Smith", "Jones", "Davis"],"LengthInSeconds": 214}'
);
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Price, Genre, CriticRating)
VALUES(
    'No One You Know', 'My Dog Spot', 'Hey Now',
    1.98, 'Country', 8.4
);
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Price, Genre)
VALUES(
    'The Acme Band', 'Look Out, World', 'The Buck Starts Here',
    0.99, 'Rock'
);
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Price, Genre, 
    Tags)
VALUES(
    'The Acme Band', 'Still In Love', 'The Buck Starts Here',
    2.47, 'Rock', 
    '{"radioStationsPlaying": ["KHCR", "KBQX", "WTNR", "WJJH"], "tourDates": { "Seattle": "20150625", "Cleveland": "20150630"}, "rotation": Heavy}'
);
"""

In [ ]:
conn = engine.connect()

try: 
    result = conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
SELECT * FROM Music;
"""

conn = engine.connect()
pd.read_sql(query, conn)

In [ ]:
query = """
SELECT * FROM Music
WHERE Artist='No One You Know';
"""

conn = engine.connect()
pd.read_sql(query, conn)

In [ ]:
query = """
SELECT * FROM Music
WHERE Artist='No One You Know' AND SongTitle LIKE 'Call%%';
"""

conn = engine.connect()
pd.read_sql(query, conn)

In [ ]:
query = """
SELECT * FROM Music
WHERE Artist='No One You Know' AND SongTitle LIKE '%%Today%%'
AND Price > 1.00;
"""

conn = engine.connect()
pd.read_sql(query, conn)

In [ ]:
query = """
UPDATE Music
SET Genre = 'Disco'
WHERE Artist = 'The Acme Band' AND SongTitle = 'Still In Love';
"""

conn = engine.connect()

try: 
    conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
SELECT * FROM Music;
"""

conn = engine.connect()
pd.read_sql(query, conn)

In [ ]:
query = """
DELETE FROM Music
WHERE Artist = 'The Acme Band' AND SongTitle = 'Look Out, World';
"""

conn = engine.connect()

try: 
    conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
SELECT * FROM Music;
"""

conn = engine.connect()
pd.read_sql(query, conn)

In [ ]:
query = """
DROP TABLE Music;
"""

conn = engine.connect()

try: 
    conn.execute(query)
except Exception as e: 
    print(e)

## Download and insert data from a csv file into the database

In [ ]:
!wget -q --show-progress https://github.com/RecoHut-Datasets/retail_general/raw/main/sample_superstore.csv

In [ ]:
!head sample_superstore.csv

In [ ]:
data = pd.read_csv('sample_superstore.csv')

In [ ]:
data.to_sql('superstore',
            con=engine,
            if_exists='replace',
            index=False,
            method='multi')

In [ ]:
conn = engine.connect()
pd.read_sql("SELECT * FROM superstore LIMIT 10", con=conn)